# 서브워드 토크나이저

기계가 학습 한 단어가 아닌 생소한 단어가 출현했을 때, OOV(Out-Of-Vocabulary) 또는 UNK(Unknown Token)이라고 합니다. 즉, 모르는 단어로 인해 문제가 생기는 것을 OOV 문제라고 합니다.<br><br>
서브워드 분리 작업은 (birthplace = birth + place) 처럼 분리하는 작업입니다. 이를 통해 OOV문제나 희귀 단어, 신조어의 문제를 완화할 수 있습니다.<br><br>
대표적인 방법론으로 BPE(Byte Pair Encoding)이 있으며, 이를 실무적으로 구현한 센텐스피스(Sentencepiece)를 배워봅니다.

## 1. BPE(Byte Pair Encoding)

BPE는 1994년에 제안된 압축 알고리즘입니다.<br>
아래와 같은 문자열이 주어졌다고 생각해봅시다.

- aaabdaaabac

위 문자열에서 가장 자주 등장하는 바이트의 쌍은 'aa' 입니다. 이 'aa' 를 Z로 치환합니다.

- ZabdZabac, Z=aa

다시 위 문자열 중에서 가장 자주 등장하는 바이트의 쌍 'ab'를 Y로 치환합니다.

- ZYdZYac, Z=aa, Y=ab

마지막으로 가장 많이 등장하는 바이트의 쌍은 'ZY'입니다. 이를 X로 치환합니다.

- XdXac, Z=aa, Y=ab, X=ZY

이제 더 이상 병합할 바이트의 쌍은 없으므로 BPE는 위의 결과를 최종으로 종료됩니다.

## 2. 자연어 처리에서의 BPE

논문 : https://arxiv.org/pdf/1508.07909.pdf

자연어 처리에서의 BPE는 서브워드 분리 알고리즘입니다. 기존에 있던 단어를 분리한다는 의미입니다. BPE를 요약하면, 글자(Character) 단위에서 점차적으로 단어 집합(Vocabulary)을 만들어 내는 Bottom Up 방식의 접근을 사용합니다. 훈련 데이터에 있는 단어들을 모든 글자(Character) 또는 유니코드(Unicode) 단위로 단어 집합을 만들고, 가장 많이 등장하는 유니그램을 하나의 유니그램으로 통합합니다.

### 1) 기존의 접근

In [1]:
# 훈련 데이터에 있는 단어와 등장 빈도 수
low = 5
lower = 2
newest = 6
widest = 3

위 훈련 데이터에는 'low'가 5회 등장했고, 'lower'가 2회 등장하였으며, 'newest'는 6회 등장하며, 'widest'가 3회 등장합니다. 위 딕셔너리로부터 추출할 수 있는 훈련 데이터의 단어 집합은 아래와 같습니다.

In [2]:
voca = ['low', 'lower', 'newest', 'widest']

위 훈련 데이터에 존재하지 않는 'lowest'라는 단어가 등장한다면 OOV문제가 발생하겠지요. 그렇다면 BPE를 적용한다면 어떻게 될까요?

### 2) BPE 알고리즘 적용

BPE 알고리즘에 의해 모든 단어들을 글자(Character) 단위로 분리합니다.<br>
l o w : 5,  l o w e r : 2,  n e w e s t : 6,  w i d e s t : 3

In [5]:
# vocabulary
voca = ['l', 'o', 'w', 'e', 'r', 'n', 'w', 's', 't', 'i', 'd']

BPE의 특징은 알고리즘의 동작을 몇 회 반복할 것인지를 사용자가 정한다는 점입니다. 여기서는 총 10회를 수행한다고 가정합니다. 즉, 가장 빈도수가 높은 유니그램의 쌍을 하나의 유니그램으로 통합하는 과정을 총 10회 반복합니다.

그 결과 아래와 같은 결과를 얻을 수 있습니다.

In [6]:
# vocabulary update!
voca_bpe = ['l', 'o', 'w', 'e', 'r', 'n', 'w', 's', 't', 'i', 'd',
            'es', 'est', 'lo', 'low', 'ne', 'new',
            'newest', 'wi', 'wid', 'widest']

이 경우 테스트 과정에서 'lowest'란 단어가 등장한다면, 기존에는 OOV에 해당하는 단어가 되었겠지만 BPE 알고리즘을 사용한 위의 단어 집합에서는 더 이상 'lowest'는 OOV가 아닙니다. 기계는 우선 'lowest'를 전부 글자 단위로 분할합니다. 즉, 'l, o, w, e, s, t'가 됩니다. 그리고 기계는 위의 단어 집합을 참고로 하여 'low'와 'est'를 찾아냅니다. 즉, 'lowest'를 기계는 'low'와 'est' 두 단어로 인코딩합니다. 그리고 이 두 단어는 둘 다 단어 집합에 있는 단어이므로 OOV가 아닙니다.

### 3) 코드 실습하기

In [7]:
import re, collections
from IPython.display import display, Markdown, Latex

In [13]:
# bpe를 몇 회 수행할 것인지 정합니다.

num_merges = 10

In [9]:
# bpe에 사용할 단어가 low, lower, newest, widest 입니다.
# </w> 는 단어의 맨 끝에 붙이는 특수 문자이며, 단어는 글자 단위로 분리합니다.

dictionary = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>':6,
         'w i d e s t </w>':3
         }

In [15]:
# bpe의 코드는 아래와 같습니다.

def get_stats(dictionary):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols) -1):
            pairs[symbols[i], symbols[i+1]] += freq
    print('현재 pair들의 빈도수: ', dict(pairs))
    return pairs

def merge_dictionary(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

In [11]:
bpe_codes = {}
bpe_codes_reverse = {}

In [16]:
for i in range(num_merges):
    display(Markdown("### Iteration {}".format(i +1)))
    pairs = get_stats(dictionary)
    best = max(pairs, key=pairs.get)
    dictionary = merge_dictionary(best, dictionary)
    
    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best
    
    print('new merge: {}'.format(best))
    print('dictionary: {}'.format(dictionary))

### Iteration 1

현재 pair들의 빈도수:  {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
new merge: ('e', 's')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


### Iteration 2

현재 pair들의 빈도수:  {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
new merge: ('es', 't')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


### Iteration 3

...

현재 pair들의 빈도수:  {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('new', 'est</w>'): 6, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('new', 'est</w>')
dictionary: {'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 9

현재 pair들의 빈도수:  {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('low', '</w>')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 10

현재 pair들의 빈도수:  {('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('w', 'i')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


In [19]:
# bpe_codes 를 출력하면 merge 했던 기록이 출력됩니다.

print(bpe_codes)

{('e', 's'): 0, ('es', 't'): 1, ('est', '</w>'): 2, ('l', 'o'): 3, ('lo', 'w'): 4, ('n', 'e'): 5, ('ne', 'w'): 6, ('new', 'est</w>'): 7, ('low', '</w>'): 8, ('w', 'i'): 9}


### 4) OOV에 대처하기

In [20]:
def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as a tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs


def encode(orig):
    """Encode word based on list of BPE merge operations, which are applied consecutively"""

    word = tuple(orig) + ('</w>',)
    display(Markdown("__word split into characters:__ <tt>{}</tt>".format(word)))

    pairs = get_pairs(word)    

    if not pairs:
        return orig

    iteration = 0
    while True:
        iteration += 1
        display(Markdown("__Iteration {}:__".format(iteration)))

        print("bigrams in the word: {}".format(pairs))
        bigram = min(pairs, key = lambda pair: bpe_codes.get(pair, float('inf')))
        print("candidate for merging: {}".format(bigram))
        if bigram not in bpe_codes:
            display(Markdown("__Candidate not in BPE merges, algorithm stops.__"))
            break
        first, second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try:
                j = word.index(first, i)
                new_word.extend(word[i:j])
                i = j
            except:
                new_word.extend(word[i:])
                break

            if word[i] == first and i < len(word)-1 and word[i+1] == second:
                new_word.append(first+second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        word = new_word
        print("word after merging: {}".format(word))
        if len(word) == 1:
            break
        else:
            pairs = get_pairs(word)

    # 특별 토큰인 </w>는 출력하지 않는다.
    if word[-1] == '</w>':
        word = word[:-1]
    elif word[-1].endswith('</w>'):
        word = word[:-1] + (word[-1].replace('</w>',''),)

    return word

In [21]:
encode("loki")

__word split into characters:__ <tt>('l', 'o', 'k', 'i', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('i', '</w>'), ('o', 'k'), ('l', 'o'), ('k', 'i')}
candidate for merging: ('l', 'o')
word after merging: ('lo', 'k', 'i', '</w>')


__Iteration 2:__

bigrams in the word: {('i', '</w>'), ('k', 'i'), ('lo', 'k')}
candidate for merging: ('i', '</w>')


__Candidate not in BPE merges, algorithm stops.__

('lo', 'k', 'i')

In [22]:
encode("lowest")

__word split into characters:__ <tt>('l', 'o', 'w', 'e', 's', 't', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('e', 's'), ('s', 't'), ('t', '</w>'), ('o', 'w'), ('w', 'e'), ('l', 'o')}
candidate for merging: ('e', 's')
word after merging: ('l', 'o', 'w', 'es', 't', '</w>')


__Iteration 2:__

bigrams in the word: {('w', 'es'), ('t', '</w>'), ('es', 't'), ('o', 'w'), ('l', 'o')}
candidate for merging: ('es', 't')
word after merging: ('l', 'o', 'w', 'est', '</w>')


...

bigrams in the word: {('lo', 'w'), ('w', 'est</w>')}
candidate for merging: ('lo', 'w')
word after merging: ('low', 'est</w>')


__Iteration 6:__

bigrams in the word: {('low', 'est</w>')}
candidate for merging: ('low', 'est</w>')


__Candidate not in BPE merges, algorithm stops.__

('low', 'est')

In [23]:
encode("lowing")

__word split into characters:__ <tt>('l', 'o', 'w', 'i', 'n', 'g', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('w', 'i'), ('g', '</w>'), ('i', 'n'), ('o', 'w'), ('n', 'g'), ('l', 'o')}
candidate for merging: ('l', 'o')
word after merging: ('lo', 'w', 'i', 'n', 'g', '</w>')


__Iteration 2:__

bigrams in the word: {('w', 'i'), ('lo', 'w'), ('i', 'n'), ('n', 'g'), ('g', '</w>')}
candidate for merging: ('lo', 'w')
word after merging: ('low', 'i', 'n', 'g', '</w>')


__Iteration 3:__

bigrams in the word: {('i', 'n'), ('n', 'g'), ('g', '</w>'), ('low', 'i')}
candidate for merging: ('i', 'n')


__Candidate not in BPE merges, algorithm stops.__

('low', 'i', 'n', 'g')

In [24]:
encode("highing")

__word split into characters:__ <tt>('h', 'i', 'g', 'h', 'i', 'n', 'g', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('g', 'h'), ('i', 'n'), ('i', 'g'), ('h', 'i'), ('n', 'g'), ('g', '</w>')}
candidate for merging: ('g', 'h')


__Candidate not in BPE merges, algorithm stops.__

('h', 'i', 'g', 'h', 'i', 'n', 'g')

## 3. Wordpiece Model

논문 : https://static.googleusercontent.com/media/research.google.com/ko//pubs/archive/37842.pdf<br>
구글이 위 Wordpiece Model을 변형하여 번역기에 사용했다는 논문 : https://arxiv.org/pdf/1609.08144.pdf

WordPiece Model은 BPE의 변형 알고리즘입니다. 이하 WPM이라 명명합니다. WPM은 BPE가 빈도수에 기반하여 가장 많이 등장한 쌍을 병합하는 것과는 달리, 병합되었을 때 코퍼스의 우도(Likelihood)를 가장 높이는 쌍을 병합합니다. 2016년의 위 논문에서 구글은 구글 번역기에서 WPM이 수행된 결과에 대해서 기술하였습니다.

WPM을 수행하기 이전의 문장: Jet makers feud over seat width with big orders at stake <br>
WPM을 수행한 결과(wordpieces): _J et _makers _fe ud _over _seat _width _with _big _orders _at _stake

Jet는 J와 et로 나누어졌으며, feud는 fe와 ud로 나누어진 것을 볼 수 있습니다. WPM은 모든 단어의 맨 앞에 _를 붙이고, 단어는 서브 워드(subword)로 통계에 기반하여 띄어쓰기로 분리합니다. 여기서 언더바 _는 문장 복원을 위한 장치입니다.

예컨대, WPM의 결과로 나온 문장을 보면, Jet → _J et와 같이 기존에 없던 띄어쓰기가 추가되어 서브 워드(subwords)들을 구분하는 구분자 역할을 하고 있습니다. 그렇다면 기존에 있던 띄어쓰기와 구분자 역할의 띄어쓰기는 어떻게 구별할까요? 이 역할을 수행하는 것이 단어들 앞에 붙은 언더바 _입니다. WPM이 수행된 결과로부터 다시 수행 전의 결과로 돌리는 방법은 현재 있는 모든 띄어쓰기를 전부 제거하고, 언더바를 띄어쓰기로 바꾸면 됩니다.

이 알고리즘은 최신 딥 러닝 모델 BERT를 훈련하기 위해서 사용되기도 하였습니다.

## 4. Unigram Language Model Tokenizer

논문 : https://arxiv.org/pdf/1804.10959.pdf

유니그램 언어 모델 토크나이저는 각각의 서브워드들에 대해서 손실(loss)을 계산합니다. 여기서 서브 단어의 손실이라는 것은 해당 서브워드가 단어 집합에서 제거되었을 경우, 코퍼스의 우도(Likelihood)가 감소하는 정도를 말합니다. 이렇게 측정된 서브워드들을 손실의 정도로 정렬하여, 최악의 영향을 주는 10~20%의 토큰을 제거합니다. 이를 원하는 단어 집합의 크기에 도달할 때까지 반복합니다.

지금까지 서브워드 토크나이징 알고리즘들에 대해서 정리해보았습니다. 이를 실무에서 사용하기 위한 패키지인 센텐스피스(SentencePiece)의 사용법에 대해서는 다음 챕터에서 학습합니다.



# 02) 센텐스피스(SentencePiece)

## 1. 센텐스피스

내부 단어 분리를 위한 유용한 패키지로 구글의 센텐스피스(Sentencepiece)가 있습니다. 구글은 BPE 알고리즘과 Unigram Language Model Tokenizer를 구현한 센텐스피스를 깃허브에 공개하였습니다.

내부 단어 분리 알고리즘을 사용하기 위해서, 데이터에 단어 토큰화를 먼저 진행한 상태여야 한다면 이 단어 분리 알고리즘을 모든 언어에 사용하는 것은 쉽지 않습니다. 영어와 달리 한국어와 같은 언어는 단어 토큰화부터가 쉽지 않기 때문입니다. 그런데, 이런 사전 토큰화 작업(pretokenization)없이 전처리를 하지 않은 데이터(raw data)에 바로 단어 분리 토크나이저를 사용할 수 있다면, 이 토크나이저는 그 어떤 언어에도 적용할 수 있는 토크나이저가 될 것입니다. 센텐스피스는 이 이점을 살려서 구현되었습니다. 센텐스피스는 사전 토큰화 작업없이 단어 분리 토큰화를 수행하므로 언어에 종속되지 않습니다.

## 2. IMDB 리뷰 토큰화하기

In [27]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

In [28]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv",
                           filename="IMDb_Reviews.csv")

('IMDb_Reviews.csv', <http.client.HTTPMessage at 0x28afbf59b88>)

In [29]:
train_df = pd.read_csv('IMDb_Reviews.csv')
train_df['review']

0        My family and I normally do not watch local mo...
1        Believe it or not, this was at one time the wo...
2        After some internet surfing, I found the "Home...
3        One of the most unheralded great works of anim...
4        It was the Sixties, and anyone with long hair ...
                               ...                        
49995    the people who came up with this are SICK AND ...
49996    The script is so so laughable... this in turn,...
49997    "So there's this bride, you see, and she gets ...
49998    Your mind will not be satisfied by this nobud...
49999    The chaser's war on everything is a weekly sho...
Name: review, Length: 50000, dtype: object

In [30]:
print('리뷰 개수 :',len(train_df)) # 리뷰 개수 출력

리뷰 개수 : 50000


총 5만개의 샘플이 존재합니다. 센텐스피스의 입력으로 사용하기 위해서 데이터프레임을 txt 파일로 저장합니다.

In [31]:
with open('imdb_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

In [32]:
# 센텐스피스로 단어 집합과 각 단어에 고유한 정수를 부여해보겠습니다.

spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

각 인자가 의미하는 바는 다음과 같습니다.
- input: 학습시킬 파일
- model_prefix: 만들어질 모델 이름
- vocab_size: 단어 집합의 크기
- model_type: 사용할 모델(unigram(default), bpe, char, word)
- max_sentence_length: 문장의 최대 길이
- pad_id, pad_piece: pad token id, 값
- unk_id, unk_piece: unknown token id, 값
- bos_id, bos_piece: begin of sentence token id, 값
- eos_id, eos_piece: end of sequence token id, 값
- user_defined_symbols: 사용자 정의 토큰

vocab 생성이 완료되면 imdb.model, imdb.vocab 파일 두개가 생성 됩니다. vocab 파일에서 학습된 서브워드들을 확인할 수 있습니다. 단어 집합의 크기를 확인하기 위해 vocab 파일을 데이터프레임에 저장해봅시다.

In [34]:
vocab_list = pd.read_csv('imdb.vocab',
                         sep='\t',
                         header=None,
                         quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

,0,1
1646,▁themselves,-1643
3638,▁dad,-3635
2771,▁Ale,-2768
3499,▁comedies,-3496
2188,nding,-2185
1088,▁talk,-1085
3630,▁discover,-3627
3655,istent,-3652
3458,face,-3455
1179,▁tal,-1176


위에서 vocab_size의 인자를 통해 단어 집합의 크기를 5,000개로 제한하였으므로 단어 집합의 크기는 5,000개입니다.

In [35]:
len(vocab_list)

5000

이제 model 파일을 로드하여 단어 시퀀스를 정수 시퀀스로 바꾸는 인코딩 작업이나 반대로 변환하는 디코딩 작업을 할 수 있습니다.

In [36]:
sp = spm.SentencePieceProcessor()
vocab_file = "imdb.model"
sp.load(vocab_file)

True

아래의 두 가지 도구를 테스트해보겠습니다.
- encode_as_pieces: 문장을 입력하면 서브 워드 시퀀스로 변환합니다.
- encode_as_ids: 문장을 입력하면 정수 시퀀스로 변환합니다.

In [37]:
lines = [
  "I didn't at all think of it this way.",
  "I have waited a long time for someone to film"
]
for line in lines:
    print(line)
    print(sp.encode_as_pieces(line))
    print(sp.encode_as_ids(line))
    print()

I didn't at all think of it this way.
['▁I', '▁didn', "'", 't', '▁at', '▁all', '▁think', '▁of', '▁it', '▁this', '▁way', '.']
[41, 623, 4950, 4926, 138, 169, 378, 30, 58, 73, 413, 4945]

I have waited a long time for someone to film
['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]



- GetPieceSize() : 단어 집합의 크기를 확인합니다.

In [38]:
sp.GetPieceSize()

5000

- idToPiece : 정수로부터 맵핑되는 서브 워드로 변환합니다.

In [42]:
sp.IdToPiece(430)

'▁character'

- PieceToId : 서브워드로부터 맵핑되는 정수로 변환합니다.

In [41]:
sp.PieceToId('▁character')

430

- DecodeIds : 정수 시퀀스로부터 문장으로 변환합니다.

In [43]:
sp.DecodeIds([41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91])

'I have waited a long time for someone to film'

In [44]:
sp.DecodePieces(['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film'])

'I have waited a long time for someone to film'

- encode : 문장으로부터 인자값에 따라서 정수 시퀀스 또는 서브워드 시퀀스로 변환 가능합니다.

In [45]:
print(sp.encode('I have waited a long time for someone to film', out_type=str))
print(sp.encode('I have waited a long time for someone to film', out_type=int))

['▁I', '▁have', '▁wa', 'ited', '▁a', '▁long', '▁time', '▁for', '▁someone', '▁to', '▁film']
[41, 141, 1364, 1120, 4, 666, 285, 92, 1078, 33, 91]


## 3. 네이버 영화 리뷰 토큰화하기

In [46]:
import pandas as pd
import sentencepiece as spm
import urllib.request
import csv

In [47]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt",
                           filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x28accc56f08>)

In [48]:
naver_df = pd.read_table('ratings.txt')
naver_df[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [49]:
print('리뷰 개수 :',len(naver_df)) # 리뷰 개수 출력

리뷰 개수 : 200000


In [50]:
print(naver_df.isnull().values.any())

True


In [51]:
naver_df = naver_df.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(naver_df.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [52]:
print('리뷰 개수 :',len(naver_df)) # 리뷰 개수 출력

리뷰 개수 : 199992


최종적으로 199,992개의 샘플을 naver_review.txt 파일에 저장한 후에 센텐스피스를 통해 단어 집합을 생성합니다.

In [53]:
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

In [54]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

In [55]:
vocab_list = pd.read_csv('naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list[:10]

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,..,0
4,영화,-1
5,▁영화,-2
6,▁이,-3
7,▁아,-4
8,...,-5
9,▁그,-6


Vocabulary 에는 unknown, 문장의 시작, 문장의 끝을 의미하는 special token이 0, 1, 2에 사용되었습니다.

In [56]:
len(vocab_list)

5000

In [57]:
sp = spm.SentencePieceProcessor()
vocab_file = "naver.model"
sp.load(vocab_file)

True

In [58]:
lines = [
  "뭐 이딴 것도 영화냐.",
  "진짜 최고의 영화입니다 ㅋㅋ",
]
for line in lines:
    print(line)
    print(sp.encode_as_pieces(line))
    print(sp.encode_as_ids(line))
    print()

뭐 이딴 것도 영화냐.
['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.']
[132, 966, 1296, 2590, 3276]

진짜 최고의 영화입니다 ㅋㅋ
['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]



In [59]:
sp.GetPieceSize()

5000

In [60]:
sp.IdToPiece(4)

'영화'

In [61]:
sp.PieceToId('영화')

4

In [62]:
sp.DecodeIds([54, 200, 821, 85])

'진짜 최고의 영화입니다 ᄏᄏ'

In [63]:
sp.DecodePieces(['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ'])

'진짜 최고의 영화입니다 ᄏᄏ'

In [64]:
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=str))
print(sp.encode('진짜 최고의 영화입니다 ㅋㅋ', out_type=int))

['▁진짜', '▁최고의', '▁영화입니다', '▁ᄏᄏ']
[54, 200, 821, 85]
